# Attempt to do the experiments

Attempt to do the experiments with the original Glass dataset.

## Part I

#### Imports

In [1]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

2023-03-30 14:25:20.718952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 14:25:20.932883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-30 14:25:20.932912: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-30 14:25:20.966220: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-30 14:25:21.980989: W tensorflow/stream_executor/platform/de

#### Opening the dataset

In [2]:
data = pd.read_csv('datasets/glass/glass.tsv', sep='\t')
data.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,target
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


#### Normalizing the data

<!-- The normalization will be simply to shrink all values to be from -1 to 1.
The minimum value will be -1; the maximum, 1.
To shrink I'll simply divide every value be the whole range of the feature. -->

I'll bring the mean to 0 by translating the data by the mean.

In [3]:
for column_name in data.columns[:-1]:
    data[column_name] -= data[column_name].mean()

In [4]:
data.describe().loc[['min', 'max']]

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,target
min,-0.007095,-2.623463,-2.745073,-1.158341,-2.816537,-0.518878,-3.509415,-0.182732,-0.059512,1.0
max,0.015525,2.436537,1.744927,2.051659,2.553463,5.691122,7.250585,2.967268,0.450488,7.0


Now I'll try to shrink the ranges

In [5]:
for column_name in data.columns[:-1]:
	length = data[column_name].max() - data[column_name].min()
	data[column_name] /= length

In [6]:
data.describe().loc[['min', 'max']]

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,target
min,-0.313677,-0.518471,-0.611375,-0.360854,-0.524495,-0.083555,-0.326154,-0.05801,-0.116691,1.0
max,0.686323,0.481529,0.388625,0.639146,0.475505,0.916445,0.673846,0.94199,0.883309,7.0


#### Renumbering the target column

The last column is not only not in successiong, but also actually ordered.

In [7]:
data['target'].is_monotonic_increasing

True

I decided to shuffle this in `train_test_split`, for now I'll just renumber them.

In [8]:
data['target'] = data['target'].factorize()[0]

Now the data is between -1 and 1, and the mean is 0.

This normalization guarantees the algorithm will stretch each range more or less in the same time.

I also have the target column as 0, 1, 2, 3, ...


## Part II

#### Splitting the dataset

I'll split the dataset for training and testing purposes

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x = data.drop('target', axis=1)
y = data['target']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

## Part III

#### Generating the network

In [12]:
num_classes = data['target'].nunique()
num_neurons = 20
num_hidden_layers = 4

In [13]:
y_train_ohe = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes)

In [14]:
from time import time 

keras_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[x_train.shape[1]]),
])

for _ in range(num_hidden_layers):
    keras_model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))

keras_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

keras_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'], )

model_path = f'datasets/dir_path/model_{num_hidden_layers}layers_glass.h5'

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
ck = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)

start = time()
keras_model.fit(x_train, y_train_ohe, batch_size=4, epochs=100, validation_data=(x_test, y_test_ohe), verbose=2, callbacks=[ck, es])
print(f'Tempo de Treinamento: {time()-start}')

keras_model = tf.keras.models.load_model(model_path)
print('Resultado Treinamento')
keras_model.evaluate(x_train, y_train_ohe, verbose=2)

print('Resultado Teste')
keras_model.evaluate(x_test, y_test_ohe, verbose=2)

2023-03-30 14:25:25.249606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-30 14:25:25.249634: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-30 14:25:25.249657: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bolsista-HP-EliteDesk-800-G1-SFF): /proc/driver/nvidia/version does not exist
2023-03-30 14:25:25.249895: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
39/39 - 1s - loss: 1.5826 - accuracy: 0.3399 - val_loss: 1.5497 - val_accuracy: 0.4808 - 1s/epoch - 27ms/step
Epoch 2/100
39/39 - 0s - loss: 1.4659 - accuracy: 0.5229 - val_loss: 1.3946 - val_accuracy: 0.5000 - 343ms/epoch - 9ms/step
Epoch 3/100
39/39 - 0s - loss: 1.2565 - accuracy: 0.6078 - val_loss: 1.2073 - val_accuracy: 0.5192 - 150ms/epoch - 4ms/step
Epoch 4/100
39/39 - 0s - loss: 1.0956 - accuracy: 0.6144 - val_loss: 1.1154 - val_accuracy: 0.5385 - 151ms/epoch - 4ms/step
Epoch 5/100
39/39 - 0s - loss: 1.0128 - accuracy: 0.5948 - val_loss: 1.0611 - val_accuracy: 0.5769 - 151ms/epoch - 4ms/step
Epoch 6/100
39/39 - 0s - loss: 0.9763 - accuracy: 0.5752 - val_loss: 1.0547 - val_accuracy: 0.4808 - 112ms/epoch - 3ms/step
Epoch 7/100
39/39 - 0s - loss: 0.9404 - accuracy: 0.5948 - val_loss: 1.0177 - val_accuracy: 0.5769 - 119ms/epoch - 3ms/step
Epoch 8/100
39/39 - 0s - loss: 0.9099 - accuracy: 0.6536 - val_loss: 0.9983 - val_accuracy: 0.5769 - 116ms/epoch - 3ms/step
Epoch 9/10

[0.8207608461380005, 0.7115384340286255]

#### Codifying the network as a set of linear constraints

In [15]:
(mp_model, output_bounds) = codify_network(keras_model, data, 'fischetti', relax_constraints=False)

#### Loading the model

In [80]:
time_for_finding_the_explanation = {'alg': [], 'anchors': []}

**QUESTION:** Should I use `data` or `X` or `X_train` or `X_test`??

In [16]:
data_numpy = data.to_numpy()

i = 138
print('i =', i)
network_input = data_numpy[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 138
1/1 [==============================] - 0s 23ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.1906623
Network output: (<class 'numpy.int64'>) 1


#### The minimal explanation

In [17]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, num_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input1](x_0,EQ,-0.0736236009574934),
 docplex.mp.LinearConstraint[input2](x_1,EQ,-0.11135640605417947),
 docplex.mp.LinearConstraint[input3](x_2,EQ,0.17258949426910744),
 docplex.mp.LinearConstraint[input4](x_3,EQ,0.02855406124154695),
 docplex.mp.LinearConstraint[input7](x_6,EQ,-0.09659987306192763),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.05801006581494386),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.11669057867049258)]

#### The improved explanation

In [18]:
import docplex

In [19]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

In [20]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input1](x_0,EQ,-0.0736236009574934),
 docplex.mp.LinearConstraint[input2](x_1,EQ,-0.11135640605417947),
 docplex.mp.LinearConstraint[input3](x_2,EQ,0.17258949426910744),
 docplex.mp.LinearConstraint[input4](x_3,EQ,0.02855406124154695),
 docplex.mp.LinearConstraint[input7](x_6,EQ,-0.09659987306192763),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.05801006581494386),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.11669057867049258)]

Replacing $x = c$ with $c \le x \le c$:

In [21]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,-0.0736236009574934),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-0.0736236009574934),
 docplex.mp.LinearConstraint[input LE](x_1,LE,-0.11135640605417947),
 docplex.mp.LinearConstraint[input GE](x_1,GE,-0.11135640605417947),
 docplex.mp.LinearConstraint[input LE](x_2,LE,0.17258949426910744),
 docplex.mp.LinearConstraint[input GE](x_2,GE,0.17258949426910744),
 docplex.mp.LinearConstraint[input LE](x_3,LE,0.02855406124154695),
 docplex.mp.LinearConstraint[input GE](x_3,GE,0.02855406124154695),
 docplex.mp.LinearConstraint[input LE](x_6,LE,-0.09659987306192763),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-0.09659987306192763),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.05801006581494386),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.05801006581494386),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.11669057867049258),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.11669057867049258)]

Finding the bounds:

In [22]:
epsilon = 0.01

variables = testing_model.find_matching_vars('x')

for constraint in linear_constraints:
	testing_model.solve()
	print('Initial constraint:' + '\t', constraint)

	variable = constraint.lhs
	while testing_model.solution is None:
		if constraint.sense == docplex.mp.constants.ComparisonType.LE:
			if constraint.rhs.constant <= variable.ub:
				constraint.rhs += epsilon
			else:
				break
		elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
			if constraint.rhs.constant >= variable.lb:
				constraint.rhs -= epsilon
			else:
				break
		else:
			raise Exception('Constraint sense was neither LE nor GE')

		testing_model.solve()

	# Undo last operation
	if constraint.sense == docplex.mp.constants.ComparisonType.LE:
		constraint.rhs -= epsilon
	elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
		constraint.rhs += epsilon

	print('Final constraint:' + '\t', constraint)
	print()

linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

Initial constraint:	 input LE: x_0 <= -0.0736236009574934
Final constraint:	 input LE: x_0 <= -0.053623600957493404

Initial constraint:	 input GE: x_0 >= -0.0736236009574934
Final constraint:	 input GE: x_0 >= -0.31362360095749353

Initial constraint:	 input LE: x_1 <= -0.11135640605417947
Final constraint:	 input LE: x_1 <= -0.11135640605417947

Initial constraint:	 input GE: x_1 >= -0.11135640605417947
Final constraint:	 input GE: x_1 >= -0.18135640605417952

Initial constraint:	 input LE: x_2 <= 0.17258949426910744
Final constraint:	 input LE: x_2 <= 0.17258949426910744

Initial constraint:	 input GE: x_2 >= 0.17258949426910744
Final constraint:	 input GE: x_2 >= 0.17258949426910744

Initial constraint:	 input LE: x_3 <= 0.02855406124154695
Final constraint:	 input LE: x_3 <= 0.028554061241546948

Initial constraint:	 input GE: x_3 >= 0.02855406124154695
Final constraint:	 input GE: x_3 >= 0.028554061241546955

Initial constraint:	 input LE: x_6 <= -0.09659987306192763
Final constr

[docplex.mp.LinearConstraint[input LE](x_0,LE,-0.053623600957493404),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-0.31362360095749353),
 docplex.mp.LinearConstraint[input LE](x_1,LE,-0.11135640605417947),
 docplex.mp.LinearConstraint[input GE](x_1,GE,-0.18135640605417952),
 docplex.mp.LinearConstraint[input LE](x_2,LE,0.17258949426910744),
 docplex.mp.LinearConstraint[input GE](x_2,GE,0.17258949426910744),
 docplex.mp.LinearConstraint[input LE](x_3,LE,0.028554061241546948),
 docplex.mp.LinearConstraint[input GE](x_3,GE,0.028554061241546955),
 docplex.mp.LinearConstraint[input LE](x_6,LE,-0.09659987306192763),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-0.3165998730619278),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.05801006581494386),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.058010065814943855),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.11669057867049258),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.11669057867049258)]

Rewriting expressions like $c \le x \le c$ as $x = c$:

In [23]:
number_of_inputs = len(data.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_0,LE,-0.053623600957493404),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-0.31362360095749353),
 docplex.mp.LinearConstraint[input LE](x_1,LE,-0.11135640605417947),
 docplex.mp.LinearConstraint[input GE](x_1,GE,-0.18135640605417952),
 docplex.mp.LinearConstraint[input LE](x_3,LE,0.028554061241546948),
 docplex.mp.LinearConstraint[input GE](x_3,GE,0.028554061241546955),
 docplex.mp.LinearConstraint[input LE](x_6,LE,-0.09659987306192763),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-0.3165998730619278),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.05801006581494386),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.058010065814943855),
 docplex.mp.LinearConstraint[input](x_2,EQ,0.17258949426910744),
 docplex.mp.LinearConstraint[input](x_8,EQ,-0.11669057867049258)]

#### Pretty printing the improved explanation

In [76]:
def get_index_of_variable(variable: docplex.mp.dvar.Var) -> int:
	index = variable.name.split('_')[1]
	return int(index)

In [77]:
def print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	for e in explanation:
		variable = e.lhs
		index = get_index_of_variable(variable)
		feature_name = data.columns[index]
		print(feature_name, e.sense.operator_symbol, e.rhs)

SANDBOX

In [ ]:
def print_explanation_unified(explanation: list[docplex.mp.constr.LinearConstraint]):
	for e in explanation:
		variable = e.lhs
		index = get_index_of_variable(variable)
		feature_name = data.columns[index]
		print(feature_name, e.sense.operator_symbol, e.rhs)

In [67]:
lex = improved_explanation[0]
gex = improved_explanation[1]

Assert that these expressions are on the same variable

In [75]:
assert lex.lhs is gex.lhs

print(f'{lex.rhs} <= {lex.lhs} <= {gex.rhs}')

-0.053623600957493404 <= x_0 <= -0.31362360095749353


END SANDBOX

In [26]:
print_explanation(improved_explanation)

RI <= -0.053623600957493404
RI >= -0.31362360095749353
Na <= -0.11135640605417947
Na >= -0.18135640605417952
Al <= 0.028554061241546948
Al >= 0.028554061241546955
Ca <= -0.09659987306192763
Ca >= -0.3165998730619278
Ba <= -0.05801006581494386
Ba >= -0.058010065814943855
Mg == 0.17258949426910744
Fe == -0.11669057867049258


## Part IV

Now I'll use `anchors` to explain an instance.

In [27]:
from anchor import utils

#### Loading the dataset into `anchors`

I need to save `x` and `y` as csv files to use with `anchors`.

In [45]:
df = pd.concat([x_test, y_test], axis=1)
df.to_csv('tiago-experiments-attempt/test.csv', index=False)

In [48]:
d = utils.load_csv_dataset(
	data='tiago-experiments-attempt/test.csv',
	target_idx=-1,
	feature_names=['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','target'],
	skip_first=True
)

The explainer:

In [49]:
from anchor import anchor_tabular

In [50]:
explainer = anchor_tabular.AnchorTabularExplainer(
    d.class_names,
    d.feature_names,
    d.train,
    d.categorical_names)

In [51]:
predict_fn = lambda x: tf.argmax(keras_model.predict(x)[0]).numpy().reshape(1)

In [53]:
exp = explainer.explain_instance(data_numpy[i, :-1], predict_fn)

1/1 [==============================] - 0s 31ms/step


In [55]:
exp = explainer.explain_instance(data_numpy[i, :-1], predict_fn)

1/1 [==============================] - 0s 31ms/step


In [56]:
exp.names()

['-0.07 < Ca <= -0.02',
 'Fe = -0.018651362984218066',
 'Si <= 0.00',
 '-0.08 < RI <= 0.07',
 'Mg > 0.16',
 'Al <= -0.01',
 '0.01 < K <= 0.01',
 'Na > 0.07',
 'Ba = -0.023089430894308944']

In [57]:
print_explanation(improved_explanation)

RI <= -0.053623600957493404
RI >= -0.31362360095749353
Na <= -0.11135640605417947
Na >= -0.18135640605417952
Al <= 0.028554061241546948
Al >= 0.028554061241546955
Ca <= -0.09659987306192763
Ca >= -0.3165998730619278
Ba <= -0.05801006581494386
Ba >= -0.058010065814943855
Mg == 0.17258949426910744
Fe == -0.11669057867049258
